In [ ]:
question = "Do you have parking?"
question

In [ ]:
df = pd.read_csv("../input/webpage-text.csv")
df.head()

In [ ]:
def get_embedding(text, model="text-embedding-3-small"):
  text=text.replace("\n", " ")
  return client.embeddings.create(input = [text], model = model).data[0].embedding

In [ ]:
get_embedding(df['text'].iloc[0])

In [ ]:
%%time
df['embedding'] = df['text'].apply(get_embedding)

In [ ]:
%%time

df['embedding'] = de['text'].apply(get_embedding)
df.to_csv("./website-with-embeddings.csv", index=False)
df.to_pickle("./website-with-embeddings.pkl")

In [ ]:
%%time

df = pd.read_pickle("./website-with-embeddings.pkl")
df.head()

In [ ]:
question_embedding = get_embedding(question)
question, question_embedding[0:10], "..."

In [ ]:
def fn(page_embedding):
  return np.dot(page_embedding, question_embedding)
df['distance'] = df['embedding'].apply(fn)
df.head()

In [ ]:
df.sort_values('distance', ascending=False, inplace=True)
df

In [ ]:
context = df['text'].iloc[0] + "\n" + df['text'].iloc[1] + "\n" + df['text'].iloc[2] + "\n" + df['text'].iloc[3] + "\n" + df['text'].iloc[4] +"\n"
print(context)

In [ ]:
response = client.chat.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are an assistant who is helping Cambridge Health Alliance (CHA) health systems respond to patient questions. When you answer the question, use the first person to refer to Cambridge Health Alliance. "},
        {"role": "user", "content": context},
        {"role": "assistant", "content": f"Use this information from CHA's website as context to answer user questions: {context}. Please stick to this content when answering the question"},
    ],)

In [ ]:
response.choices[0].message.content

# Now making it repeatable

In [ ]:
def query(question):
  question_embedding = get_embedding(question)

  def fn(page_embedding):
    return np.dot(page_embedding, question_embedding)
  distance_series = df['embedding'].apply(fn)

  top_four = distance_series.sort_values(ascending=False).index[0:4]

  text_series = df.loc[top_four]['text']
  context = "\n\n".join(text_series)

  response = client.chat.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are an assistant who is helping Cambridge Health Alliance (CHA) health systems respond to patient questions. When you answer the question, use the first person to refer to Cambridge Health Alliance. "},
        {"role": "user", "content": context},
        {"role": "assistant", "content": f"Use this information from CHA's website as context to answer user questions: {context}. Please stick to this content when answering the question"},
    ],)

  return response.choices[0].message.content